In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arogo-final/final_99999.csv
/kaggle/input/arogo-final/final_median.csv


In [3]:

df2 = pd.read_csv("/kaggle/input/arogo-final/final_median.csv")

In [4]:
df2 = df2.drop(columns=['Unnamed: 0'])

In [5]:
df2.sample(5)

,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
122780,0,27.0,5,0,1,1.0,3.0,5.69,5.0,3.0,3,2,16,0,5.0,2.0,0,0
61616,0,53.0,9,1,10,3.0,2.0,7.77,3.0,5.0,4,1,20,1,1.0,5.0,0,0
90911,0,49.0,12,1,12,3.0,5.0,7.77,3.0,2.0,4,3,13,0,6.0,4.0,1,0
123344,1,27.0,20,1,4,3.0,5.0,7.77,3.0,1.0,3,1,2,1,5.0,5.0,1,1
117784,1,48.0,18,1,7,3.0,4.0,7.77,3.0,2.0,2,3,14,0,0.0,2.0,0,0


In [6]:
X = df2.drop('Depression', axis=1)
y = df2['Depression']

In [ ]:
# ANN Implementation

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test).flatten()
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(128, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='elu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stop, lr_scheduler], class_weight=class_weight_dict)
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)


Epoch 1/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8483 - loss: 0.5748 - val_accuracy: 0.9100 - val_loss: 0.3851 - learning_rate: 5.0000e-04
Epoch 2/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9034 - loss: 0.3634 - val_accuracy: 0.9043 - val_loss: 0.2988 - learning_rate: 5.0000e-04
Epoch 3/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9029 - loss: 0.2868 - val_accuracy: 0.9058 - val_loss: 0.2545 - learning_rate: 5.0000e-04
Epoch 4/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9036 - loss: 0.2529 - val_accuracy: 0.9104 - val_loss: 0.2390 - learning_rate: 5.0000e-04
Epoch 5/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9056 - loss: 0.2419 - val_accuracy: 0.9099 - val_loss: 0.2268 - learning_rate: 5.0000e-04
Epoch 6/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9054 - loss: 0.2394 - val_accuracy: 0.9004 - val_loss: 0.2412 - learning_rate: 5.0000e-04
Epoch 7/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/st

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test).flatten()

classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(128, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(64, activation='elu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    
    Dense(32, activation='elu'),
    BatchNormalization(),
    
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stop], class_weight=class_weight_dict)
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)
y_pred = (model.predict(X_test) > 0.5).astype("int32")
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


Epoch 1/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8807 - loss: 0.4821 - val_accuracy: 0.9047 - val_loss: 0.2622
Epoch 2/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9038 - loss: 0.2517 - val_accuracy: 0.9189 - val_loss: 0.2111
Epoch 3/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9043 - loss: 0.2300 - val_accuracy: 0.9181 - val_loss: 0.2234
Epoch 4/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9067 - loss: 0.2349 - val_accuracy: 0.8888 - val_loss: 0.3153
Epoch 5/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9057 - loss: 0.2369 - val_accuracy: 0.9126 - val_loss: 0.2354
Epoch 6/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9063 - loss: 0.2326 - val_accuracy: 0.8875 - val_loss: 0.3135
Epoch 7/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9098 - loss: 0.2292 - val_accuracy: 0.9217 - val_loss: 0.2036
Epoch 8/50
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9086 - loss: 0.2287 - 

In [ ]:
# Hyperparameter tuning for ANN

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test).flatten()
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(units=hp.Choice('units1', values=[256, 512, 1024]), activation=hp.Choice('act1', values=['elu','relu']), kernel_regularizer=l2(hp.Choice('l2_1', values=[0.001, 0.0001]))))
    model.add(BatchNormalization())
    model.add(Dense(units=hp.Choice('units2', values=[64, 128, 256]), activation=hp.Choice('act2', values=['elu','relu']), kernel_regularizer=l2(hp.Choice('l2_2', values=[0.001, 0.0001]))))
    model.add(BatchNormalization())
    model.add(Dense(units=hp.Choice('units3', values=[32, 64, 128]), activation=hp.Choice('act3', values=['elu','relu']), kernel_regularizer=l2(hp.Choice('l2_3', values=[0.001, 0.0001]))))
    model.add(BatchNormalization())
    model.add(Dense(units=hp.Choice('units4', values=[16, 32, 64]), activation=hp.Choice('act4', values=['elu','relu'])))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    lr = hp.Choice('lr', values=[0.001, 0.0005, 0.0001])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=50, factor=3, directory='hyper_dir', project_name='hyper_tuning')
stop_early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early], class_weight=class_weight_dict)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[stop_early], class_weight=class_weight_dict)
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)
y_pred = (model.predict(X_test) > 0.5).astype("int32")
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


Trial 90 Complete [00h 02m 31s]
val_accuracy: 0.9233953356742859

Best val_accuracy So Far: 0.9367488622665405
Total elapsed time: 01h 16m 05s
Epoch 1/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8883 - loss: 0.5574 - val_accuracy: 0.9066 - val_loss: 0.2697
Epoch 2/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9069 - loss: 0.2506 - val_accuracy: 0.9234 - val_loss: 0.1996
Epoch 3/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9099 - loss: 0.2273 - val_accuracy: 0.9181 - val_loss: 0.2076
Epoch 4/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9103 - loss: 0.2213 - val_accuracy: 0.9063 - val_loss: 0.2465
Epoch 5/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9107 - loss: 0.2215 - val_accuracy: 0.9211 - val_loss: 0.2011
Epoch 6/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9092 - loss: 0.2239 - val_accuracy: 0.9202 - val_loss: 0.2082
Epoch 7/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9120 - loss: 0.2180 - v

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

model = xgb.XGBClassifier(
    objective='binary:logistic',
    booster='gbtree',
    learning_rate=0.1,
    n_estimators=100,
    max_depth=6,
    min_child_weight=1,
    gamma=0,
    subsample=1,
    colsample_bytree=1,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')


In [14]:
print(best_hps)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

search_spaces = {
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'n_estimators': Integer(50, 500),
    'max_depth': Integer(3, 10),
    'min_child_weight': Integer(1, 10),
    'gamma': Real(0, 0.5),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0),
    'reg_alpha': Real(0, 1.0),
    'reg_lambda': Real(0, 1.0)
}

model = xgb.XGBClassifier(
    objective='binary:logistic',
    booster='gbtree',
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    scoring='accuracy',
    cv=3,
    n_iter=30,
    random_state=42,
    verbose=0
)

opt.fit(X_train, y_train)

print("Best parameters:", opt.best_params_)
print("Best cross-validation accuracy: {:.4f}".format(opt.best_score_))

y_pred = opt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test F1 Score: {f1:.4f}')
